In [ ]:
import numpy as np
from numpy import array as arr
from numpy import zeros as zarr

import pandas as pd
from pandas import DataFrame as df
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import matplotlib as mpl
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
mpl.rcParams["font.serif"] = "CMU serif"
mpl.rcParams["mathtext.fontset"] = "custom"
mpl.rcParams["mathtext.rm"] = "CMU serif"
mpl.rcParams["mathtext.it"] = "CMU serif:italic"
mpl.rcParams["mathtext.bf"] = "CMU serif:bold"
mpl.rcParams["font.family"] = "serif"

# import pylandau
# from scipy import curve_fit
import scipy
import uncertainties as unc
from uncertainties import unumpy as unp
from uncertainties.unumpy import uarray as uarr
from uncertainties.unumpy import nominal_values as val
from uncertainties.unumpy import std_devs as dev
from uncertainties import ufloat as uf
# import ROOT

# import my plotting stuff
import sys
sys.path.append('./PythonHelpers/')
from PlotLib import Plotting
import PlotLib.Histogramming as Hist
import PlotLib.Plotting as Plot
from CSVimporter.runSettings import load_settings
from CSVimporter.importer import load_run
from CSVimporter.importer import dict_to_arr

import ROOT



def df_newcol(dframe, column, value, loc=-1):
    if column in dframe.columns and type(dframe.loc[0, column]) == type(value):
        print("WARNING: column", column, " already exists and has the same type")
        return
    if loc == -1:
        loc = len(dframe.columns)
    elif column in dframe.columns:
        loc = len(dframe.columns)
    try:
        try:
            dframe.insert(loc, column, pd.Series([value.copy()]*dframe.size, dtype=object))
        except:
            dframe.insert(loc, column, pd.Series([value.Clone()]*dframe.size, dtype=object))
    except:
        dframe.insert(loc, column, pd.Series([value]*dframe.size, dtype=object))
        
def df_query(frame, args, col, single=True):
    if type(args) == dict:
        query_str = " & ".join(["{:s}=={:n}".format(key, value) for key, value in args.items()])
    elif type(args) == list:
        query_str = " & ".join(["{:s}=={:n}".format(key, value) for key, value in args])
    elif type(args) == str:
        query_str = args
    index = frame.query(query_str).index
    if single:
        if len(index) != 1:
            raise Exception("Query returned {:n} entries".format(len(index)))
        index = index[0]
    return frame.at[index, col]
        
def get_hist_TH1(hHist):
    binN = hHist.GetNbinsX()
    bins = zarr(binN+1)    
    vals = zarr(binN)
    bins[0] = hHist.GetBinLowEdge(1)
    for i in range(binN):
        bins[i+1] = hHist.GetBinLowEdge(i+2)
        vals[i] = hHist.GetBinContent(i+1)
    return vals, bins

def truncate_hist(vals, bins, interval=0.95):
    vals_norm = vals / np.sum(vals)
    bin_centers = (bins[1:]+bins[:-1])/2
    Mean = np.sum(vals_norm*bin_centers)
    Stop = (1-interval)/2

    S = 0
    for i in range(len(vals)):
        S += vals_norm[i]
        if S > Stop:
            i_low = i
            break
    S = 1
    for i in range(len(vals)-1, -1, -1):
        S -= vals_norm[i]
        if S < 1-Stop:
            i_up = i
            break
    
    # need to include bin i_up, thus return up to i_up+1
    return vals[i_low:i_up+1], bins[i_low:i_up+2]

def hist_rebin(vals, bins, factor):
    # factor = how many bins to combine
    
    if len(vals) % factor != 0:
        print("WARNING: Number of bins not divisible by factor, losing " + str(len(vals) % factor) + " bin(s)")
    binN = len(vals)
    new_binN = binN//factor
    new_bins = zarr(new_binN+1)
    new_vals = zarr(new_binN)
    
    new_bins = bins[::factor]
    for i in range(new_binN):
        new_vals[i] = np.sum(vals[i*factor:(i+1)*factor])
    return new_vals, new_bins

In [ ]:
thr = 220
runID = 190
SEC = 0.5
MTP = 6

FilePath = "/home/jona/DESY/analysis_TB/output/scan_tracking/"
FileName = FilePath+str(runID) + "_thr{:n}".format(thr) + "_trkPlanes{:n}".format(MTP) + "_trackingGood" + ".root"
# output/scan_tracking/190_thr220_TrkCut150_TrkPlanes6_AssocCut60_SEC0.5.root

rFile = ROOT.TFile(FileName)
if not rFile:
    print("File not found")
    rFile.Close()

# Get the association residual histogram
trunc_interval = 0.99
nameDir = f"AnalysisDUT/DSO9254A_0/local_residuals"
rDir = rFile.Get(nameDir)
if not rDir:
    print(FileName)
    raise Exception("Dir.", nameDir, "not found in root file")
else:
    # Get residual-x histogram & fit to get gaussian
    nameObj = "residualsX"
    hResidualX = rDir.Get(nameObj)
    if not hResidualX:
        raise Exception("Object", nameObj, "not found in root file at dir.", nameDir)
    else:
        valsX_preTrunc, binsX_preTrunc = get_hist_TH1(hResidualX)
        valsX, binsX = truncate_hist(valsX_preTrunc, binsX_preTrunc, trunc_interval)
        bin_centers = (binsX[1:]+binsX[:-1])/2
        residualX_mean = np.sum(valsX*bin_centers)/np.sum(valsX)
        residualX_std = np.sqrt(np.sum(valsX*(bin_centers-residualX_mean)**2)/np.sum(valsX))
        
        
    # Get residual-y histogram & fit to get gaussian
    nameObj = "residualsY"
    hResidualY = rDir.Get(nameObj)
    if not hResidualY:
        raise Exception("Object", nameObj, "not found in root file at dir.", nameDir)
    else:
        valsY_preTrunc, binsY_preTrunc = get_hist_TH1(hResidualY)
        valsY, binsY = truncate_hist(valsY_preTrunc, binsY_preTrunc, trunc_interval)
        bin_centers = (binsY[1:]+binsY[:-1])/2
        residualY_mean = np.sum(valsY*bin_centers)/np.sum(valsY)
        residualY_std = np.sqrt(np.sum(valsY*(bin_centers-residualY_mean)**2)/np.sum(valsY))
        
rFile.Close()

runSettings = load_settings(runID)

cut_sigma = 5
cuts = cut_sigma * np.array([residualX_std, residualY_std])

print(f"cuts at {cut_sigma} sigma of largest residuals")
print("x:", cuts[0])
print("y:", cuts[1])

In [ ]:
fig, ax = Hist.create_fig(1,1, figsize=(4.5,3))

# x
vals, bins = hist_rebin(valsX_preTrunc, binsX_preTrunc, 10)
Color="black"
Label = r"Residuals $x$"
ax.stairs(vals/np.sum(vals), bins, color=Color, label=Label, lw=0.9)
ax.stairs(vals/np.sum(vals), bins, color=Color, fill=True, alpha=0.1, lw=0.9)
ax.axvline(residualX_mean-residualX_std, color=Color, linestyle="--", label=r"$\sigma\,_{X,"+f" trunc. {trunc_interval*100:n}"+r"\%}$", lw=0.9)
ax.axvline(residualX_mean+residualX_std, color=Color, linestyle="--", lw=0.9)

vals, bins = hist_rebin(valsY_preTrunc, binsY_preTrunc, 10)
Color="darkred"
Label = r"Residuals $y$"
ax.stairs(vals/np.sum(vals), bins, color=Color, label=Label, lw=0.9)
ax.stairs(vals/np.sum(vals), bins, color=Color, fill=True, alpha=0.1, lw=0.9)
ax.axvline(residualY_mean-residualY_std, color=Color, linestyle="--", label=r"$\sigma\,_{Y,"+f" trunc. {trunc_interval*100:n}"+r"\%}$", lw=0.9)
ax.axvline(residualY_mean+residualY_std, color=Color, linestyle="--", lw=0.9)

ax.set_xlim(-50,50)
binWidth = bins[1]-bins[0]
Hist.finalize(runSettings, fig, ax, 
    title="Track to DUT cluster\nassociation residuals", ER1=True,
    xlabel=r"track pos. $-$ DUT cluster centre pos. / um",
    ylabel= f"relative frequency / {binWidth}" + r"$\,$um",
    param_dict={"campaign":True, "sample":True, "ER1Param":True, "recoParam":True, "fontsize":7, "thr":thr, "edgeCut":None, "minTrkPlanes":MTP},
    grid=False, legend_loc="upper right")

# Hist.finalize(runSettings, fig, ax, title="Track to DUT cluster\nassociation residuals", xlabel=r"track pos. $-$ DUT cluster centre pos. / um", ylabel=f"relative frequency / {binWidth} um", param_narrow=True, grid=False, thr=thr, ER1=False)

Hist.savefig(fig, f"cuts/association_residuals_{runID}_thr{thr}")